In [87]:
%matplotlib inline
import pandas as pd
import os
import numpy as np

In [88]:
pdX = pd.read_pickle(os.path.join('..', '..', 'data', 'our_df2.pickled'))
y = pd.read_pickle(os.path.join('..', '..', 'data', 'ytrain.pkl'))

# y has one more predicion - trim it
y = y[0:-1]

In [89]:
print(pdX.shape)
print(len(y)) # y is 1-dim

(17253, 337)
17253


In [90]:
# preprocess data here
# our approach has left some Nones, let's do a second pass ;)

# def clean(x):
#     if x is None:
#         return hash(x)
#     else:
#         return x


# pdX = pdX.applymap(clean)
pdX.fillna(0, inplace=True)
# from sklearn.preprocessing import scale
# pdX = pdX.apply(lambda col:scale(col))
print("done")

done


In [94]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xg

# suggested params
param = {
    'bst:eta' : [0.002, 0.02],
    'bst:subsample' : [0.5,1],
    'bst:mim_child_weight' : [0.1,10],
    'bst:max_depth' : [2,30],
    'bst:gamma' : [0,1],
    'objective':'binary:logistic'
}

X_fit, X_eval, y_fit, y_eval = train_test_split(
    pdX, y, test_size=0.3, random_state=42
)
# X_fit = pdX
# y_fit = y
# X_eval = pdX
# y_eval = y

# print("fit shape: ", X_fit.shape)
# print("eval shape: ", X_eval.shape)
# print("fit 1s:", len(filter(lambda x: x == 1, y_fit)))
# print("eval 1s:", len(filter(lambda x: x == 1, y_eval)))

# X = xg.DMatrix(np.array(pdX))
tb = xg.XGBClassifier(learning_rate=0.05).fit(X_fit, y_fit)
# tb = xg.XGBClassifier(learning_rate=0.05)
# tb.fit(X_fit, y_fit, eval_set=[(X_eval, y_eval)], eval_metric='logloss', verbose=True)
# print tb.evals_result()
y_pred = tb.predict_proba(X_eval)

In [95]:
print len(y_pred)
# print f1_score(y_eval, y_pred)
print len(filter(lambda x: x, map(lambda x: x[1] > 0.07, y_pred)))

5176
332


In [96]:
# auc
from sklearn.metrics import roc_auc_score
y_true = y_eval
y_scores = map(lambda x: x[1], y_pred)
roc_auc_score(y_true, y_scores)

0.73859620555655803

In [9]:
import pickle
with open(os.path.join('..','data', 'y_pred.pickle'), 'w') as f:
    pickle.dump(y_pred,f)

In [14]:
# pd.read_pickle(os.path.join('..','data', 'y_pred.pickle'))